In [1]:
import sys

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()

grid = Grid.cartesian_from_polygon(
  polygon=polygon,
  n_rows=200,
  n_cols=200,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')

In [3]:
from morpheus.project.types.geometry import LineString

linestring_within = LineString(
  type='LineString',
  coordinates=[
    (13.923, 50.965),
    (13.924, 50.965),
  ]
)

In [4]:
%%time
from morpheus.project.types.discretization.spatial import ActiveCells

ac = ActiveCells.from_linestring(linestring=linestring_within, grid=grid)


CPU times: user 1.99 s, sys: 22.4 ms, total: 2.01 s
Wall time: 1.74 s
